In [1]:
import pandas as pd
df = pd.read_csv('/content/diabetes.csv')

In [2]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
statistics_table = df.describe()
statistics_table

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [4]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


# 1)KNN & Normalisation

In [5]:
df_knn= df.copy()

In [6]:
from sklearn.preprocessing import StandardScaler

# Initialize StandardScaler
scaler = StandardScaler()

# Separate features (excluding Outcome) and the target variable
features = df_knn.drop('Outcome', axis=1)
target = df_knn['Outcome']

# Fit and transform the features using Z-score normalization
scaled_features = scaler.fit_transform(features)

# Assign the scaled features back to df_knn while keeping the Outcome column unchanged
df_knn[features.columns] = scaled_features

# Ensure the Outcome column remains unchanged
df_knn['Outcome'] = target


## no selection

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score

# Assuming df_knn is already defined, scaled, and has the 'Outcome' column as the target variable
# Separate features and target variable
X = df_knn.drop('Outcome', axis=1)  # Features
y = df_knn['Outcome']  # Target variable

# Lists to store the final results for each outer loop iteration
outer_accuracies = []
outer_f1_scores = []

# Outer loop: Perform 20 different train/test splits
for outer_i in range(20):
    # Split data into train (80%) and test (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

    # Lists to store results from inner cross-validation loop
    inner_accuracies = []
    inner_f1_scores = []

    # Inner loop: Perform 20 cross-validations on the 80% training set
    for inner_i in range(20):
        # Split training data into 80% train and 20% validation for cross-validation
        X_train_temp, X_val, y_train_temp, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=None)

        # Train KNN model on the cross-validation training data
        clf = KNeighborsClassifier()
        clf.fit(X_train_temp, y_train_temp)

        # Evaluate on the validation set
        y_val_pred = clf.predict(X_val)
        accuracy = accuracy_score(y_val, y_val_pred)
        f1 = f1_score(y_val, y_val_pred)

        # Store the cross-validation metrics
        inner_accuracies.append(accuracy)
        inner_f1_scores.append(f1)

    # Calculate the mean of the inner cross-validation metrics
    mean_inner_accuracy = np.mean(inner_accuracies)
    mean_inner_f1 = np.mean(inner_f1_scores)
    print(f'Outer Run {outer_i+1} - Mean Cross-Validation Accuracy: {mean_inner_accuracy:.4f}, Mean F1 Score: {mean_inner_f1:.4f}')

    # Train on the entire train set (80%) and evaluate on the test set
    final_model = KNeighborsClassifier()
    final_model.fit(X_train, y_train)
    y_test_pred = final_model.predict(X_test)

    # Calculate test metrics
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)

    # Store test metrics for this outer iteration
    outer_accuracies.append(test_accuracy)
    outer_f1_scores.append(test_f1)

    print(f'Outer Run {outer_i+1} - Test Accuracy: {test_accuracy:.4f}, Test F1 Score: {test_f1:.4f}')

# Calculate and print the overall mean test metrics across all outer iterations
accuracy_knn0_no = np.mean(outer_accuracies)
f1_knn0_no = np.mean(outer_f1_scores)

print(f'\nOverall Mean Test Accuracy from 20 Outer Splits: {accuracy_knn0_no:.4f}')
print(f'Overall Mean Test F1 Score from 20 Outer Splits: {f1_knn0_no:.4f}')


Outer Run 1 - Mean Cross-Validation Accuracy: 0.7264, Mean F1 Score: 0.5831
Outer Run 1 - Test Accuracy: 0.7273, Test F1 Score: 0.5435
Outer Run 2 - Mean Cross-Validation Accuracy: 0.7280, Mean F1 Score: 0.5458
Outer Run 2 - Test Accuracy: 0.7338, Test F1 Score: 0.6095
Outer Run 3 - Mean Cross-Validation Accuracy: 0.7386, Mean F1 Score: 0.5923
Outer Run 3 - Test Accuracy: 0.7532, Test F1 Score: 0.5581
Outer Run 4 - Mean Cross-Validation Accuracy: 0.7207, Mean F1 Score: 0.5730
Outer Run 4 - Test Accuracy: 0.7013, Test F1 Score: 0.5660
Outer Run 5 - Mean Cross-Validation Accuracy: 0.7301, Mean F1 Score: 0.5770
Outer Run 5 - Test Accuracy: 0.6818, Test F1 Score: 0.5421
Outer Run 6 - Mean Cross-Validation Accuracy: 0.7110, Mean F1 Score: 0.5373
Outer Run 6 - Test Accuracy: 0.7208, Test F1 Score: 0.5905
Outer Run 7 - Mean Cross-Validation Accuracy: 0.7553, Mean F1 Score: 0.5990
Outer Run 7 - Test Accuracy: 0.6623, Test F1 Score: 0.5185
Outer Run 8 - Mean Cross-Validation Accuracy: 0.7305, M

## ReliefF

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from skrebate import ReliefF

# Assume df_knn is already defined, contains all the mentioned variables, and is scaled

# Separate features and target variable
X = df_knn.drop('Outcome', axis=1)
y = df_knn['Outcome']

# Initialize lists to store outer loop metrics
outer_accuracies = []
outer_f1_scores = []

# Outer loop: Split data 20 times into 80% train and 20% test
for outer_loop in range(20):
    # Split data into 80% train and 20% test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

    # Implement ReliefF on the 80% training data to select top 5 features
    relief = ReliefF(n_features_to_select=5, n_neighbors=100)
    X_train_relief = relief.fit_transform(X_train.values, y_train.values)

    # Obtain feature names of the selected features
    selected_features = X_train.columns[relief.top_features_[:5]]
    print(f"Outer Loop {outer_loop + 1}: Selected Features - {selected_features}")

    # Update X_train and X_test with the selected features
    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]

    # Initialize lists to store inner loop metrics
    inner_accuracies = []
    inner_f1_scores = []

    # Inner loop: Cross-validation within the training set
    for inner_loop in range(20):
        # Split the training data further into 80% training and 20% validation
        X_train_temp, X_val, y_train_temp, y_val = train_test_split(X_train_selected, y_train, test_size=0.2, random_state=None)

        # Train and evaluate the model
        clf = KNeighborsClassifier()
        clf.fit(X_train_temp, y_train_temp)
        y_val_pred = clf.predict(X_val)

        # Calculate and store accuracy and F1 score for this inner loop
        inner_accuracies.append(accuracy_score(y_val, y_val_pred))
        inner_f1_scores.append(f1_score(y_val, y_val_pred))

    # Calculate mean accuracy and F1 score from the inner cross-validation loop
    mean_inner_accuracy = np.mean(inner_accuracies)
    mean_inner_f1 = np.mean(inner_f1_scores)
    print(f"Outer Loop {outer_loop + 1}: Inner Mean Accuracy - {mean_inner_accuracy:.4f}, Inner Mean F1 - {mean_inner_f1:.4f}")

    # Append inner loop metrics to outer loop lists
    outer_accuracies.append(mean_inner_accuracy)
    outer_f1_scores.append(mean_inner_f1)

    # Train the model on the full training set and evaluate on the test set
    final_model = KNeighborsClassifier()
    final_model.fit(X_train_selected, y_train)
    y_test_pred = final_model.predict(X_test_selected)

    # Calculate and store metrics for this outer loop
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    print(f"Outer Loop {outer_loop + 1}: Test Accuracy - {test_accuracy:.4f}, Test F1 - {test_f1:.4f}")
accuracy_knn0_r = np.mean(outer_accuracies)
f1_knn0_r = np.mean(outer_f1_scores)
# Display overall mean metrics across all outer loops
print(f"\nOverall Mean Accuracy (Outer Loop): {np.mean(outer_accuracies):.4f}")
print(f"Overall Mean F1 (Outer Loop): {np.mean(outer_f1_scores):.4f}")


Outer Loop 1: Selected Features - Index(['Glucose', 'Age', 'Pregnancies', 'SkinThickness', 'BMI'], dtype='object')
Outer Loop 1: Inner Mean Accuracy - 0.7447, Inner Mean F1 - 0.6123
Outer Loop 1: Test Accuracy - 0.7208, Test F1 - 0.5825
Outer Loop 2: Selected Features - Index(['Glucose', 'Age', 'Pregnancies', 'SkinThickness', 'BMI'], dtype='object')
Outer Loop 2: Inner Mean Accuracy - 0.7707, Inner Mean F1 - 0.6410
Outer Loop 2: Test Accuracy - 0.6623, Test F1 - 0.5357
Outer Loop 3: Selected Features - Index(['Glucose', 'Age', 'SkinThickness', 'Pregnancies', 'BMI'], dtype='object')
Outer Loop 3: Inner Mean Accuracy - 0.7350, Inner Mean F1 - 0.5835
Outer Loop 3: Test Accuracy - 0.7597, Test F1 - 0.6667
Outer Loop 4: Selected Features - Index(['Glucose', 'Age', 'Pregnancies', 'BMI', 'SkinThickness'], dtype='object')
Outer Loop 4: Inner Mean Accuracy - 0.7175, Inner Mean F1 - 0.5921
Outer Loop 4: Test Accuracy - 0.7727, Test F1 - 0.6237
Outer Loop 5: Selected Features - Index(['Glucose', 

## mrMR

In [14]:
!pip install pymrmr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pymrmr: filename=pymrmr-0.1.11-cp310-cp310-linux_x86_64.whl size=390762 sha256=67b88c70a31a97cc0b52beaa64e65db9724895f8f42e02e35364d514e82baf0c
  Stored in directory: /root/.cache/pip/wheels/46/ae/55/4a2479c5f0de7eb363fe970cb18e4a750e03e4e63b1b5c2005
Successfully built pymrmr


In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
import pymrmr

# Assuming df_knn is already defined and scaled with the 'Outcome' column as the target variable

# Initialize lists to hold the overall accuracies and F1 scores from the outer loop
outer_accuracies = []
outer_f1_scores = []

# Outer loop for random train-test splits
for outer_i in range(20):
    # Split data into 80% train and 20% test
    X = df_knn.drop('Outcome', axis=1)
    y = df_knn['Outcome']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

    # Apply mRMR to select the top 5 features from the training data
    X_train_mr = X_train.copy()
    X_train_mr['Outcome'] = y_train
    selected_features = pymrmr.mRMR(X_train_mr, 'MIQ', 5)  # Minimum Redundancy Feature Selection

    # Filter out 'Outcome' from the selected features
    selected_features = [feature for feature in selected_features if feature != 'Outcome']

    # Select the columns corresponding to the selected features
    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]

    # Lists to store the accuracies and F1 scores for the inner loop
    inner_accuracies = []
    inner_f1_scores = []

    # Inner loop for cross-validation within the training set
    for inner_i in range(20):
        # Split the 80% train set into 80% train and 20% validation
        X_train_temp, X_val, y_train_temp, y_val = train_test_split(X_train_selected, y_train, test_size=0.2, random_state=None)

        # Train and evaluate the KNN classifier
        clf = KNeighborsClassifier()
        clf.fit(X_train_temp, y_train_temp)
        y_val_pred = clf.predict(X_val)

        # Record metrics for the inner loop
        accuracy = accuracy_score(y_val, y_val_pred)
        f1 = f1_score(y_val, y_val_pred)

        inner_accuracies.append(accuracy)
        inner_f1_scores.append(f1)

    # Calculate and store the mean metrics from the inner loop
    mean_inner_accuracy = np.mean(inner_accuracies)
    mean_inner_f1 = np.mean(inner_f1_scores)
    outer_accuracies.append(mean_inner_accuracy)
    outer_f1_scores.append(mean_inner_f1)

    # Test the model on the hold-out test set using selected features
    final_model = KNeighborsClassifier()
    final_model.fit(X_train_selected, y_train)
    y_test_pred = final_model.predict(X_test_selected)

    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)

    print(f'Outer Loop {outer_i + 1}: Test Accuracy = {test_accuracy:.4f}, Test F1 = {test_f1:.4f}')

# Calculate and save the overall mean metrics across all outer loops
accuracy_knn0_mr = np.mean(outer_accuracies)
f1_knn0_mr = np.mean(outer_f1_scores)

# Display the overall mean metrics
print(f"\nOverall Mean Accuracy (Outer Loop with mRMR): {accuracy_knn0_mr:.4f}")
print(f"Overall Mean F1 (Outer Loop with mRMR): {f1_knn0_mr:.4f}")


Outer Loop 1: Test Accuracy = 0.6623, Test F1 = 0.4468
Outer Loop 2: Test Accuracy = 0.6429, Test F1 = 0.4554
Outer Loop 3: Test Accuracy = 0.7208, Test F1 = 0.5981
Outer Loop 4: Test Accuracy = 0.6558, Test F1 = 0.4952
Outer Loop 5: Test Accuracy = 0.6558, Test F1 = 0.4952
Outer Loop 6: Test Accuracy = 0.7468, Test F1 = 0.5301
Outer Loop 7: Test Accuracy = 0.7468, Test F1 = 0.5979
Outer Loop 8: Test Accuracy = 0.7078, Test F1 = 0.5872
Outer Loop 9: Test Accuracy = 0.7208, Test F1 = 0.5376
Outer Loop 10: Test Accuracy = 0.6688, Test F1 = 0.4950
Outer Loop 11: Test Accuracy = 0.7532, Test F1 = 0.6042
Outer Loop 12: Test Accuracy = 0.6623, Test F1 = 0.5185
Outer Loop 13: Test Accuracy = 0.6429, Test F1 = 0.4762
Outer Loop 14: Test Accuracy = 0.7078, Test F1 = 0.5714
Outer Loop 15: Test Accuracy = 0.6234, Test F1 = 0.4314
Outer Loop 16: Test Accuracy = 0.6234, Test F1 = 0.4314
Outer Loop 17: Test Accuracy = 0.7013, Test F1 = 0.5741
Outer Loop 18: Test Accuracy = 0.7468, Test F1 = 0.6422
O

## SVM-RFE-G

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_selection import RFE
from sklearn.svm import LinearSVC

# Initialize lists to hold the overall accuracies and F1 scores from the outer loop
outer_accuracies = []
outer_f1_scores = []

# Outer loop for random train-test splits
for outer_i in range(20):
    # Split data into 80% train and 20% test
    X = df_knn.drop('Outcome', axis=1)
    y = df_knn['Outcome']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

    # Apply SVM-RFE with Linear kernel to select the top 5 features from the training data
    svc = LinearSVC(max_iter=10000)  # Use LinearSVC with a higher iteration limit
    rfe = RFE(estimator=svc, n_features_to_select=5)
    rfe.fit(X_train, y_train)

    # Get the selected features based on RFE ranking
    selected_features = X_train.columns[rfe.support_]

    # Select the columns corresponding to the selected features
    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]

    # Lists to store the accuracies and F1 scores for the inner loop
    inner_accuracies = []
    inner_f1_scores = []

    # Inner loop for cross-validation within the training set
    for inner_i in range(20):
        # Split the 80% train set into 80% train and 20% validation
        X_train_temp, X_val, y_train_temp, y_val = train_test_split(X_train_selected, y_train, test_size=0.2, random_state=None)

        # Train and evaluate the KNN classifier
        clf = KNeighborsClassifier()
        clf.fit(X_train_temp, y_train_temp)
        y_val_pred = clf.predict(X_val)

        # Record metrics for the inner loop
        accuracy = accuracy_score(y_val, y_val_pred)
        f1 = f1_score(y_val, y_val_pred)

        inner_accuracies.append(accuracy)
        inner_f1_scores.append(f1)

    # Calculate and store the mean metrics from the inner loop
    mean_inner_accuracy = np.mean(inner_accuracies)
    mean_inner_f1 = np.mean(inner_f1_scores)
    outer_accuracies.append(mean_inner_accuracy)
    outer_f1_scores.append(mean_inner_f1)

    # Test the model on the hold-out test set using selected features
    final_model = KNeighborsClassifier()
    final_model.fit(X_train_selected, y_train)
    y_test_pred = final_model.predict(X_test_selected)

    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)

    print(f'Outer Loop {outer_i + 1}: Test Accuracy = {test_accuracy:.4f}, Test F1 = {test_f1:.4f}')

# Calculate and save the overall mean metrics across all outer loops
accuracy_knn0_svmrfe = np.mean(outer_accuracies)
f1_knn0_svmrfe = np.mean(outer_f1_scores)

# Display the overall mean metrics
print(f"\nOverall Mean Accuracy (Outer Loop with SVM-RFE-G): {accuracy_knn0_svmrfe:.4f}")
print(f"Overall Mean F1 (Outer Loop with SVM-RFE-G): {f1_knn0_svmrfe:.4f}")


Outer Loop 1: Test Accuracy = 0.7338, Test F1 = 0.5773
Outer Loop 2: Test Accuracy = 0.7338, Test F1 = 0.5773
Outer Loop 3: Test Accuracy = 0.7143, Test F1 = 0.5417
Outer Loop 4: Test Accuracy = 0.7338, Test F1 = 0.5859
Outer Loop 5: Test Accuracy = 0.7338, Test F1 = 0.6168
Outer Loop 6: Test Accuracy = 0.6948, Test F1 = 0.4946
Outer Loop 7: Test Accuracy = 0.6494, Test F1 = 0.4808
Outer Loop 8: Test Accuracy = 0.7403, Test F1 = 0.6154
Outer Loop 9: Test Accuracy = 0.6883, Test F1 = 0.5636
Outer Loop 10: Test Accuracy = 0.7273, Test F1 = 0.5625
Outer Loop 11: Test Accuracy = 0.7727, Test F1 = 0.6392
Outer Loop 12: Test Accuracy = 0.7208, Test F1 = 0.5567
Outer Loop 13: Test Accuracy = 0.6623, Test F1 = 0.5185
Outer Loop 14: Test Accuracy = 0.6948, Test F1 = 0.6050
Outer Loop 15: Test Accuracy = 0.7273, Test F1 = 0.6250
Outer Loop 16: Test Accuracy = 0.7468, Test F1 = 0.6286
Outer Loop 17: Test Accuracy = 0.7532, Test F1 = 0.6545
Outer Loop 18: Test Accuracy = 0.7273, Test F1 = 0.5625
O